In [1]:
from google.colab.drive import mount
mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd drive/MyDrive/UIRS_sem7

/content/drive/MyDrive/UIRS_sem7


In [1]:
!pip install pyyaml==5.1
!pip install torch==1.8.0+cu101 torchvision==0.9.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
#install old version of pytorch since detectron2 hasn't released packages for pytorch 1.9

     ------------------------------------ 215.1/215.1 kB 773.2 kB/s eta 0:00:00
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ---------------------------------------- 1.3/1.3 GB 459.7 kB/s eta 0:00:00
     ---------------------------------------- 1.6/1.6 MB 251.4 kB/s eta 0:00:00


In [3]:
!pip install cython

In [6]:
!git clone https://github.com/facebookresearch/detectron2.git detectron_repo

Cloning into 'detectron_repo'...


In [7]:
!pip install -q -e detectron_repo


In [8]:
%cd detectron_repo


D:\-V\Project\UIRS_sem7\detectron_repo


In [9]:
!python setup.py build develop

running build
running build_py
copying detectron2\__init__.py -> build\lib.win-amd64-cpython-37\detectron2
creating build\lib.win-amd64-cpython-37\tools
copying tools\analyze_model.py -> build\lib.win-amd64-cpython-37\tools
copying tools\benchmark.py -> build\lib.win-amd64-cpython-37\tools
copying tools\convert-torchvision-to-d2.py -> build\lib.win-amd64-cpython-37\tools
copying tools\lazyconfig_train_net.py -> build\lib.win-amd64-cpython-37\tools
copying tools\lightning_train_net.py -> build\lib.win-amd64-cpython-37\tools
copying tools\plain_train_net.py -> build\lib.win-amd64-cpython-37\tools
copying tools\train_net.py -> build\lib.win-amd64-cpython-37\tools
copying tools\visualize_data.py -> build\lib.win-amd64-cpython-37\tools
copying tools\visualize_json_results.py -> build\lib.win-amd64-cpython-37\tools
copying tools\__init__.py -> build\lib.win-amd64-cpython-37\tools
creating build\lib.win-amd64-cpython-37\detectron2\checkpoint
copying detectron2\checkpoint\c2_model_loading.py -

C:\Users\User\anaconda3\envs\grienv\lib\site-packages\torch\utils\cpp_extension.py:369: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
  warnings.warn(msg.format('we could not find ninja.'))
C:\Users\User\anaconda3\envs\grienv\lib\site-packages\torch\utils\cpp_extension.py:304: UserWarning: Error checking compiler version for cl: [WinError 2] Не удается найти указанный файл
  warnings.warn(f'Error checking compiler version for {compiler}: {error}')
C:\Users\User\anaconda3\envs\grienv\lib\site-packages\setuptools\command\easy_install.py:147: EasyInstallDeprecationWarning: easy_install command is deprecated. Use build and pip and other standards-based tools.
  EasyInstallDeprecationWarning,
C:\Users\User\anaconda3\envs\grienv\lib\site-packages\setuptools\command\install.py:37: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  

In [4]:
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html
# After this step it will ask you to restart the runtime, please do it.

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html


ERROR: Could not find a version that satisfies the requirement detectron2 (from versions: none)
ERROR: No matching distribution found for detectron2


In [1]:
import torch
assert torch.__version__.startswith("1.8") 
import torchvision
import cv2

In [2]:
import os
import numpy as np
import json
import random
import matplotlib.pyplot as plt
%matplotlib inline

from detectron2.structures import BoxMode
from detectron2.data import DatasetCatalog, MetadataCatalog


In [5]:
def get_data_dicts(directory, classes):
    dataset_dicts = []
    for filename in [file for file in os.listdir(directory) if file.endswith('.json')]:
        json_file = os.path.join(directory, filename)
        with open(json_file) as f:
            img_anns = json.load(f)

        record = {}
        
        filename = os.path.join(directory, img_anns["imagePath"])
        
        record["file_name"] = filename
        record["height"] = 1080
        record["width"] = 1920
      
        annos = img_anns["shapes"]
        objs = []
        for anno in annos:
            px = [a[0] for a in anno['points']] # x coord
            py = [a[1] for a in anno['points']] # y-coord
            poly = [(x, y) for x, y in zip(px, py)] # poly for segmentation
            poly = [p for x in poly for p in x]

            obj = {
                "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
                "bbox_mode": BoxMode.XYXY_ABS,
                "segmentation": [poly],
                "category_id": classes.index(anno['label']),
                "iscrowd": 0
            }
            objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts

In [6]:
classes = ['asphalt', 'tile', 'snowy_dirt', 'snow', 'dirty_tile', 'icy_tile']

data_path = 'DATA/'

for d in ["train", "test"]:
    DatasetCatalog.register(
        "category_" + d, 
        lambda d=d: get_data_dicts(data_path+d, classes)
    )
    MetadataCatalog.get("category_" + d).set(thing_classes=classes, thing_colors = [
        (255,255,0),(0,0,255),(0,255,0),(255,0,255), (180,165,0), (187,132,156)]) #Done by me
  
    

microcontroller_metadata = MetadataCatalog.get("category_train")


In [7]:
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import ColorMode, Visualizer

In [8]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("category_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 5000
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 6


In [9]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)

[11/12 06:22:47 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

model_final_f10217.pkl: 178MB [00:03, 49.7MB/s]                           
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}
roi_heads.mask_head.predictor.{bias, weight}


In [ ]:
trainer.train()

[11/12 06:23:02 d2.engine.train_loop]: Starting training from iteration 0
[11/12 06:23:25 d2.utils.events]:  eta: 1:33:41  iter: 19  total_loss: 3.63  loss_cls: 2.02  loss_box_reg: 0.08792  loss_mask: 0.6944  loss_rpn_cls: 0.7145  loss_rpn_loc: 0.1292  time: 1.1512  data_time: 0.0486  lr: 4.9953e-06  max_mem: 4820M
[11/12 06:23:48 d2.utils.events]:  eta: 1:33:00  iter: 39  total_loss: 3.249  loss_cls: 1.895  loss_box_reg: 0.07332  loss_mask: 0.6929  loss_rpn_cls: 0.5014  loss_rpn_loc: 0.1149  time: 1.1323  data_time: 0.0210  lr: 9.9902e-06  max_mem: 4821M
[11/12 06:24:13 d2.utils.events]:  eta: 1:34:51  iter: 59  total_loss: 3.037  loss_cls: 1.578  loss_box_reg: 0.09183  loss_mask: 0.6901  loss_rpn_cls: 0.5664  loss_rpn_loc: 0.1715  time: 1.1741  data_time: 0.0245  lr: 1.4985e-05  max_mem: 4821M
[11/12 06:24:37 d2.utils.events]:  eta: 1:36:35  iter: 79  total_loss: 2.423  loss_cls: 1.187  loss_box_reg: 0.1004  loss_mask: 0.6874  loss_rpn_cls: 0.2127  loss_rpn_loc: 0.1451  time: 1.1843 

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.4 
cfg.DATASETS.TEST = ("test", )
predictor = DefaultPredictor(cfg)

In [ ]:
test_dataset_dicts = get_data_dicts(data_path+'test', classes)

In [ ]:
class MyVisualizer(Visualizer):
    def _jitter(self, color ):
        return color


In [ ]:

img = cv2.imread(data_path+'test/173.jpg')
outputs = predictor(img)
v = MyVisualizer(img[:, :, ::-1],
                metadata=microcontroller_metadata, 
                scale=0.8, 
                instance_mode=ColorMode.SEGMENTATION, # removes the colors of unsegmented pixels
)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
plt.figure(figsize = (16, 9))
plt.imshow(cv2.cvtColor(v.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB))
plt.show()